## Data Extraction and Storage
The Aim of this notebook is to extract relevant data that pertains the scope of this project.


## Feature Selection
From the <strong>Vehicular_coordination_Visualization Notebook </strong> it is clear that the datasets contain a significant number of information that might not be necessary in the prediction of lane change maneuvres.

For that reason, necessary columns are selcted which are to be used in the training and evaluation of the model.

It is important to note that some of the techniques used in this notebook have been inferenced from the <a href = "https://github.com/RobertKrajewski/highD-dataset"> HighD dataset tools</a> that are provided together with the data.


In [2]:
# Importing requisite libraries and classes.
import pandas as pd
import numpy as np
import pickle
import os
import random

#### Columns to extract from the dataset files are as provided

In [3]:
# TRACK FILE
BBOX = "bbox"
FRAMES = "frames"
FRAME = "frame"
TRACK_ID = "id"
X = "x"
Y = "y"
WIDTH = "width"
HEIGHT = "height"
X_VELOCITY = "xVelocity"
Y_VELOCITY = "yVelocity"
X_ACCELERATION = "xAcceleration"
Y_ACCELERATION = "yAcceleration"
FRONT_SIGHT_DISTANCE = "frontSightDistance"
BACK_SIGHT_DISTANCE = "backSightDistance"
DHW = "dhw"
THW = "thw"
TTC = "ttc"
PRECEDING_X_VELOCITY = "precedingXVelocity"
PRECEDING_ID = "precedingId"
FOLLOWING_ID = "followingId"
LEFT_PRECEDING_ID = "leftPrecedingId"
LEFT_ALONGSIDE_ID = "leftAlongsideId"
LEFT_FOLLOWING_ID = "leftFollowingId"
RIGHT_PRECEDING_ID = "rightPrecedingId"
RIGHT_ALONGSIDE_ID = "rightAlongsideId"
RIGHT_FOLLOWING_ID = "rightFollowingId"
LANE_ID = "laneId"

# STATIC FILE
INITIAL_FRAME = "initialFrame"
FINAL_FRAME = "finalFrame"
NUM_FRAMES = "numFrames"
CLASS = "class"
DRIVING_DIRECTION = "drivingDirection"
TRAVELED_DISTANCE = "traveledDistance"
MIN_X_VELOCITY = "minXVelocity"
MAX_X_VELOCITY = "maxXVelocity"
MEAN_X_VELOCITY = "meanXVelocity"
MIN_DHW = "minDHW"
MIN_THW = "minTHW"
MIN_TTC = "minTTC"
NUMBER_LANE_CHANGES = "numLaneChanges"

# RECORDING META
ID = "id"
FRAME_RATE = "frameRate"
LOCATION_ID = "locationId"
SPEED_LIMIT = "speedLimit"
MONTH = "month"
WEEKDAY = "weekDay"
START_TIME = "startTime"
DURATION = "duration"
TOTAL_DRIVEN_DISTANCE = "totalDrivenDistance"
TOTAL_DRIVEN_TIME = "totalDrivenTime"
N_VEHICLES = "numVehicles"
N_CARS = "numCars"
N_TRUCKS = "numTrucks"
UPPER_LANE_MARKINGS = "upperLaneMarkings"
LOWER_LANE_MARKINGS = "lowerLaneMarkings"


## Function Definations
    These Functions help read data and extract it in a dictionary format for easier handling in the preceeding steps.

In [4]:
def read_tracks(track_csv_path):
    """
    This method reads a provided track file from the dataset

    :param arguments: input file path to the track file information
    
    :Return: a dictionary containing all tracks with the Id as the key
    """
    
    # Read the csv file
    df = pd.read_csv(track_csv_path)

    # Use groupby to aggregate track info. Less error prone than iterating over the data.
    grouped = df.groupby([TRACK_ID], sort=False)
    # pre-allocate an empty dictionary
    tracks = {}
    current_track = 0
    
    # Extract the necessary track information !!! All info used currently
    
    for group_id, rows in grouped:
        # create the vehicles bounding box. Defines the vehicle dimensions.
        bounding_boxes = np.transpose(np.array([rows[X].values,
                                                rows[Y].values,
                                                rows[WIDTH].values,
                                                rows[HEIGHT].values]))
        
        
        tracks[np.int64(group_id)] = {
            FRAME: rows[FRAME].values,
            X: rows[X].values,
            Y: rows[Y].values,
            BBOX: bounding_boxes,
            X_VELOCITY: rows[X_VELOCITY].values,
            Y_VELOCITY: rows[Y_VELOCITY].values,
            X_ACCELERATION: rows[X_ACCELERATION].values,
            Y_ACCELERATION: rows[Y_ACCELERATION].values,
            FRONT_SIGHT_DISTANCE: rows[FRONT_SIGHT_DISTANCE].values,
            BACK_SIGHT_DISTANCE: rows[BACK_SIGHT_DISTANCE].values,
            THW: rows[THW].values,
            TTC: rows[TTC].values,
            DHW: rows[DHW].values,
            PRECEDING_X_VELOCITY: rows[PRECEDING_X_VELOCITY].values,
            PRECEDING_ID: rows[PRECEDING_ID].values,
            FOLLOWING_ID: rows[FOLLOWING_ID].values,
            LEFT_FOLLOWING_ID: rows[LEFT_FOLLOWING_ID].values,
            LEFT_ALONGSIDE_ID: rows[LEFT_ALONGSIDE_ID].values,
            LEFT_PRECEDING_ID: rows[LEFT_PRECEDING_ID].values,
            RIGHT_FOLLOWING_ID: rows[RIGHT_FOLLOWING_ID].values,
            RIGHT_ALONGSIDE_ID: rows[RIGHT_ALONGSIDE_ID].values,
            RIGHT_PRECEDING_ID: rows[RIGHT_PRECEDING_ID].values,
            LANE_ID: rows[LANE_ID].values
        }
        current_track = current_track + 1
        
    print("Finished Extracting Tracks Data")
    return tracks


          
def read_tracks_meta(tracks_metadata_path):
    """
    Reads the XX_tracksMeta.csv file from the dataset

    :param arguments: input file path to the tracksMeta file
    
    :return: the static dictionary - the key is the track_id and the value is the corresponding data for this track
    """
    # Read the csv file
    df = pd.read_csv(tracks_metadata_path)

    # Initialize the static_dictionary
    metadata_dictionary = {}

    # Iterate over all rows of the csv because we need to create the bounding boxes for each row
    for i_row in range(df.shape[0]):
        track_id = int(df[TRACK_ID][i_row])
        metadata_dictionary[track_id] = {TRACK_ID: track_id,
                                       WIDTH: float(df[WIDTH][i_row]),
                                       HEIGHT: float(df[HEIGHT][i_row]),
                                       INITIAL_FRAME: int(df[INITIAL_FRAME][i_row]),
                                       FINAL_FRAME: int(df[FINAL_FRAME][i_row]),
                                       NUM_FRAMES: int(df[NUM_FRAMES][i_row]),
                                       CLASS: str(df[CLASS][i_row]),
                                       DRIVING_DIRECTION: float(df[DRIVING_DIRECTION][i_row]),
                                       TRAVELED_DISTANCE: float(df[TRAVELED_DISTANCE][i_row]),
                                       MIN_X_VELOCITY: float(df[MIN_X_VELOCITY][i_row]),
                                       MAX_X_VELOCITY: float(df[MAX_X_VELOCITY][i_row]),
                                       MEAN_X_VELOCITY: float(df[MEAN_X_VELOCITY][i_row]),
                                       MIN_TTC: float(df[MIN_TTC][i_row]),
                                       MIN_THW: float(df[MIN_THW][i_row]),
                                       MIN_DHW: float(df[MIN_DHW][i_row]),
                                       NUMBER_LANE_CHANGES: int(
                                           df[NUMBER_LANE_CHANGES][i_row])
                                       }
          
    print("Finished Extracting Tracks_Metadata")
    return metadata_dictionary


def read_recording_meta(recording_meta_path):
    """
    This method reads the Recording meta file from the dataset : XX_recordingMeta.csv

    :param arguments: input path to the XX_recordingMeta.csv file
    
    :return: the meta dictionary containing the general information of the video
    """
          
    # Read the csv file
    df = pd.read_csv(recording_meta_path)

    # Extract the information from the files and store the to below variable      
    extracted_meta_dictionary = {ID: int(df[ID][0]),
                                 FRAME_RATE: int(df[FRAME_RATE][0]),
                                 LOCATION_ID: int(df[LOCATION_ID][0]),
                                 SPEED_LIMIT: float(df[SPEED_LIMIT][0]),
                                 MONTH: str(df[MONTH][0]),
                                 WEEKDAY: str(df[WEEKDAY][0]),
                                 START_TIME: str(df[START_TIME][0]),
                                 DURATION: float(df[DURATION][0]),
                                 TOTAL_DRIVEN_DISTANCE: float(df[TOTAL_DRIVEN_DISTANCE][0]),
                                 TOTAL_DRIVEN_TIME: float(df[TOTAL_DRIVEN_TIME][0]),
                                 N_VEHICLES: int(df[N_VEHICLES][0]),
                                 N_CARS: int(df[N_CARS][0]),
                                 N_TRUCKS: int(df[N_TRUCKS][0]),
                                 UPPER_LANE_MARKINGS: np.fromstring(df[UPPER_LANE_MARKINGS][0], sep=";"),
                                 LOWER_LANE_MARKINGS: np.fromstring(df[LOWER_LANE_MARKINGS][0], sep=";")}
          
    print("Finished Extracting Recording_Metadata")
    return extracted_meta_dictionary


In [ ]:
# Extract the data and store in a pickle file for easier retrieval and use

In [5]:
FRAME_TAKEN = 50  # number of states to construct features
FRAME_BEFORE = 12  # frame taken before the lane change
FRAME_BEFORE_FLAG = False # use FRAME_BEFORE or not

def run(number):
    '''
    This function runs the data processing code and output to pickle files
    '''
    # Read the three different files
    tracks_csv = read_tracks("data/" + number + "_tracks.csv")
    tracks_meta = read_tracks_meta("data/" + number + "_tracksMeta.csv")
    recording_meta = read_recording_meta("data/" + number + "_recordingMeta.csv")
    
    
    # Obtain the lane changing cars and lane keeping cars
    lane_changing_ids = []
    lane_keeping_ids = []
    
    for key in tracks_meta:
        if(tracks_meta[key][NUMBER_LANE_CHANGES] > 0):
            lane_changing_ids.append(key)
        else:
            lane_keeping_ids.append(key)

            
    # get the lane information
    lanes_info = {}
    # Two lane are removed since the top most and bottom most files are not usually used
    lane_num = len(recording_meta[UPPER_LANE_MARKINGS]) + \
        len(recording_meta[LOWER_LANE_MARKINGS]) - 2
    # Associating the lanes as per the context, by eliminating the unused lanes
    if lane_num == 4:
        # 4 lanes. get rid of lanes 1 and 4
        lanes_info[2] = recording_meta[UPPER_LANE_MARKINGS][0]
        lanes_info[3] = recording_meta[UPPER_LANE_MARKINGS][1]
        lanes_info[5] = recording_meta[LOWER_LANE_MARKINGS][0]
        lanes_info[6] = recording_meta[LOWER_LANE_MARKINGS][1]
        lane_width = ((lanes_info[3] - lanes_info[2]) +
                      (lanes_info[6] - lanes_info[5])) / 2
    elif lane_num == 6:
        # 6 lanes. get rid of lanes 1,5
        lanes_info[2] = recording_meta[UPPER_LANE_MARKINGS][0]
        lanes_info[3] = recording_meta[UPPER_LANE_MARKINGS][1]
        lanes_info[4] = recording_meta[UPPER_LANE_MARKINGS][2]
        lanes_info[6] = recording_meta[LOWER_LANE_MARKINGS][0]
        lanes_info[7] = recording_meta[LOWER_LANE_MARKINGS][1]
        lanes_info[8] = recording_meta[LOWER_LANE_MARKINGS][2]
        lane_width = ((lanes_info[3] - lanes_info[2]) + (lanes_info[4] - lanes_info[3]) +
                      (lanes_info[7] - lanes_info[6]) + (lanes_info[8] - lanes_info[7])) / 4
    elif lane_num == 7:
        # 7 lanes: track 58 ~ 60 . get rid of lanes 1,6
        lanes_info[2] = recording_meta[UPPER_LANE_MARKINGS][0]
        lanes_info[3] = recording_meta[UPPER_LANE_MARKINGS][1]
        lanes_info[4] = recording_meta[UPPER_LANE_MARKINGS][2]
        lanes_info[5] = recording_meta[UPPER_LANE_MARKINGS][3]
        lanes_info[7] = recording_meta[LOWER_LANE_MARKINGS][0]
        lanes_info[8] = recording_meta[LOWER_LANE_MARKINGS][1]
        lanes_info[9] = recording_meta[LOWER_LANE_MARKINGS][2]
        lane_width = ((lanes_info[3] - lanes_info[2]) + (lanes_info[4] - lanes_info[3]) + (
            lanes_info[5] - lanes_info[4]) + (lanes_info[8] - lanes_info[7]) + (lanes_info[9] - lanes_info[8])) / 5
    else:
        print("Error: Unrecognized input file number -", number)

        
        
    def determine_lane_exist(cur_lane):
        '''
        return: left_exist, right_exist 
        Find the existence of neighbor lanes through a very unpleasant hardcoded manner.
        1 Overtake Left.
        0 Overtake right
        Otherwise Return 2, stay in your current lane
        '''
        if lane_num == 4:
            if cur_lane == 2 or cur_lane == 6:
                return 1, 0
            else:
                return 0, 1
        elif lane_num == 6:
            if cur_lane == 2 or cur_lane == 8:
                return 1, 0
            elif cur_lane == 3 or cur_lane == 7:
                return 1, 1
            else:
                return 0, 1
        elif lane_num == 7:
            if cur_lane == 2 or cur_lane == 9:
                return 1, 0
            elif cur_lane == 3 or cur_lane == 4 or cur_lane == 8:
                return 1, 1
            else:
                return 0, 1

    def construct_features(i, frame_num, original_lane):
        '''
        Construct all the features for the RNN to train:
        Here is the list:
        Difference of the ego car’s Y position and the lane center: ΔY
        Ego car’s X velocity: Vx
        Ego car’s Y velocity: Vy
        Ego car’s X acceleration: Ax
        Ego car’s Y acceleration: Ay
        Ego car type: T
        TTC of preceding car: TTCp
        TTC of following car: TTCf
        TTC of left preceding car: TTClp
        TTC of left alongside car: TTCla
        TTC of left following car: TTClf
        TTC of right preceding car: TTCrp
        TTC of right alongside car: TTCra
        TTC of right following car: TTCrf
        '''
        going = 0  # 1 left, 2 right
        if lane_num == 4:
            if original_lane == 2 or original_lane == 3:
                going = 1
            else:
                going = 2
        else:
            if original_lane == 2 or original_lane == 3 or original_lane == 4 or original_lane == 5:
                going = 1
            else:
                going = 2
        cur_feature = {}
        cur_feature["left_lane_exist"], cur_feature["right_lane_exist"] = determine_lane_exist(
            original_lane)

        # We need to consider the fact that right/left are different for top/bottom lanes.
        # top lanes are going left      <----
        # bottom lanes are going right  ---->
        # left -> negative, right -> positive
        if going == 1:
            cur_feature["delta_y"] = tracks_csv[i][Y][frame_num] - \
                lanes_info[original_lane]  # up
            cur_feature["y_velocity"] = -tracks_csv[i][Y_VELOCITY][frame_num]
            cur_feature["y_acceleration"] = - \
                tracks_csv[i][Y_ACCELERATION][frame_num]
        else:
            cur_feature["delta_y"] = lanes_info[original_lane] - \
                tracks_csv[i][Y][frame_num]  # down
            cur_feature["y_velocity"] = tracks_csv[i][Y_VELOCITY][frame_num]
            cur_feature["y_acceleration"] = tracks_csv[i][Y_ACCELERATION][frame_num]

        cur_feature["x_velocity"] = tracks_csv[i][X_VELOCITY][frame_num]
        cur_feature["x_acceleration"] = tracks_csv[i][X_ACCELERATION][frame_num]
        cur_feature["car_type"] = 1 if tracks_meta[i][CLASS] == "Car" else -1

        def calculate_ttc(target_car_id):
        
            """
            Calculate time to collision of target car and current car
            """
            if target_car_id != 0:
                target_frame = tracks_meta[i][INITIAL_FRAME] + \
                    frame_num - tracks_meta[target_car_id][INITIAL_FRAME]
                target_x = tracks_csv[target_car_id][X][target_frame]
                cur_x = tracks_csv[i][X][frame_num]
                target_v = tracks_csv[target_car_id][X_VELOCITY][target_frame]
                cur_v = tracks_csv[i][X_VELOCITY][frame_num]
                if target_v == cur_v:
                    return 99999
                if going == 1:
                    # going left (up)
                    if cur_x > target_x:
                        ttc = (cur_x - target_x) / (cur_v - target_v)
                    else:
                        ttc = (target_x - cur_x) / (target_v - cur_v)
                else:
                    # going right (down)
                    if cur_x > target_x:
                        ttc = (cur_x - target_x) / (target_v - cur_v)
                    else:
                        ttc = (target_x - cur_x) / (cur_v - target_v)
                if ttc < 0:
                    return 99999
                else:
                    return ttc
            else:
                return 99999

        # surrounding cars info
        cur_feature["preceding_ttc"] = calculate_ttc(
            tracks_csv[i][PRECEDING_ID][frame_num])
        cur_feature["following_ttc"] = calculate_ttc(
            tracks_csv[i][FOLLOWING_ID][frame_num])
        cur_feature["left_preceding_ttc"] = calculate_ttc(
            tracks_csv[i][LEFT_PRECEDING_ID][frame_num])
        cur_feature["left_alongside_ttc"] = calculate_ttc(
            tracks_csv[i][LEFT_ALONGSIDE_ID][frame_num])
        cur_feature["left_following_ttc"] = calculate_ttc(
            tracks_csv[i][LEFT_FOLLOWING_ID][frame_num])
        cur_feature["right_preceding_ttc"] = calculate_ttc(
            tracks_csv[i][RIGHT_PRECEDING_ID][frame_num])
        cur_feature["right_alongside_ttc"] = calculate_ttc(
            tracks_csv[i][RIGHT_ALONGSIDE_ID][frame_num])
        cur_feature["right_following_ttc"] = calculate_ttc(
            tracks_csv[i][RIGHT_FOLLOWING_ID][frame_num])

        ret = tuple(cur_feature.values())
        return ret

    def detect_lane_change(lane_center, cur_y, lane_width, car_height):
        delta_y = abs(lane_center - cur_y)
        relative_diff = delta_y / car_height
        if(relative_diff < 0.5):
            return True
        else:
            return False

    def determine_change_direction(ori_laneId, new_laneId):
        '''
        return 1 upon left change
        return 2 upon right change
        '''
        if lane_num == 4:
            if (ori_laneId == 2 and new_laneId == 3) or (ori_laneId == 6 and new_laneId == 5):
                return 1
            else:
                return 2
        else:
            # left:
            if (ori_laneId == 2 and new_laneId == 3) or (ori_laneId == 4 and new_laneId == 5) \
                or (ori_laneId == 3 and new_laneId == 4) or (ori_laneId == 7 and new_laneId == 6) \
                    or (ori_laneId == 8 and new_laneId == 7) or (ori_laneId == 9 and new_laneId == 8):
                return 1
            else:
                return 2

    # list of list of features
    result = []

    for i in lane_changing_ids:
        # for each car:
        last_boundary = 0
        # list of (starting index, ending index, direction)
        changing_tuple_list = []
        # 1. determine the frame we want to use
        for frame_num in range(1, len(tracks_csv[i][FRAME])):
            if tracks_csv[i][LANE_ID][frame_num] != tracks_csv[i][LANE_ID][frame_num-1]:
                original_lane = tracks_csv[i][LANE_ID][frame_num-1]
                new_lane = tracks_csv[i][LANE_ID][frame_num]
                direction = determine_change_direction(original_lane, new_lane)
                # calculate the starting frame
                starting_change = frame_num - 1
                while starting_change > last_boundary:
                    if detect_lane_change(lanes_info[original_lane], tracks_csv[i][Y][starting_change], lane_width, tracks_meta[i][HEIGHT]):
                        break
                    starting_change -= 1
                # calculate the starting and ending frame
                if FRAME_BEFORE_FLAG:
                    starting_point = starting_change - FRAME_TAKEN - FRAME_BEFORE
                    ending_point = starting_change - FRAME_BEFORE
                else:
                    starting_point = starting_change - FRAME_TAKEN
                    ending_point = starting_change
                if starting_point > last_boundary:
                    changing_tuple_list.append(
                        (starting_point, ending_point, direction))
                last_boundary = frame_num
        # add those frames' features
        for pair in changing_tuple_list:
            # for each lane change i nstance
            cur_change = []
            start_idx = pair[0]
            end_idx = pair[1]
            direction = pair[2]
            original_lane = tracks_csv[i][LANE_ID][start_idx]
            # continue for out of boundary cases
            if original_lane not in lanes_info:
                continue
            for frame_num in range(start_idx, end_idx):
                # construct the object
                cur_change.append(construct_features(
                    i, frame_num, original_lane))
            # add to the result
            result.append((cur_change, direction))

    change_num = len(result)

    if len(lane_keeping_ids) > len(result):
        # make the lane keeping size the same as lane changing
        lane_keeping_ids = random.sample(lane_keeping_ids, len(result))

    for i in lane_keeping_ids:
        cur_change = []
        original_lane = tracks_csv[i][LANE_ID][0]
        fail = False
        for frame_num in range(1, FRAME_TAKEN+1):
            try:
                cur_change.append(construct_features(
                    i, frame_num, original_lane))
            except:
                # handle exception where the total frame is less than FRAME_TAKEN
                fail = True
                break
        if not fail:
            result.append((cur_change, 0))

    return result, change_num


# The result of this is a dataset with the required data necessary to train the model

##### Data processing Functions Overview
The following functions Provide a brief overview of the functions implemented above and what each function does 
<blockquote>
    <ol>
       <li> determine_lane_exist </li>
        <li> construct_features</li>
        <li> calculate_ttc</li>
        <li> detect_lane_change</li>
            Determines whether a vehicle retains its lane, changes to left lane or changes to right lane.
        <li> determine_change_direction</li>
    </ol>
    </blockquote>

In [6]:
# Create an Output file location if none exists
if not os.path.exists("output"):
    os.makedirs("output")

    
total_change = 0
for i in range(1, 61):
    number = "{0:0=2d}".format(i)
    result, change_num = run(number)
    total_change += change_num
    print("total changes:", total_change)

    filename = "output/result" + number + ".pickle"
    f = open(filename, 'wb')
    pickle.dump(result, f)
    print("Successfully write to:", filename)
    f.close()


Finished Extracting Tracks Data
Finished Extracting Tracks_Metadata
Finished Extracting Recording_Metadata
total changes: 84
Successfully write to: output/result01.pickle
Finished Extracting Tracks Data
Finished Extracting Tracks_Metadata
Finished Extracting Recording_Metadata
total changes: 166
Successfully write to: output/result02.pickle
Finished Extracting Tracks Data
Finished Extracting Tracks_Metadata
Finished Extracting Recording_Metadata
total changes: 243
Successfully write to: output/result03.pickle
Finished Extracting Tracks Data
Finished Extracting Tracks_Metadata
Finished Extracting Recording_Metadata
total changes: 418
Successfully write to: output/result04.pickle
Finished Extracting Tracks Data
Finished Extracting Tracks_Metadata
Finished Extracting Recording_Metadata
total changes: 594
Successfully write to: output/result05.pickle
Finished Extracting Tracks Data
Finished Extracting Tracks_Metadata
Finished Extracting Recording_Metadata
total changes: 793
Successfully wr

total changes: 6924
Successfully write to: output/result48.pickle
Finished Extracting Tracks Data
Finished Extracting Tracks_Metadata
Finished Extracting Recording_Metadata
total changes: 7061
Successfully write to: output/result49.pickle
Finished Extracting Tracks Data
Finished Extracting Tracks_Metadata
Finished Extracting Recording_Metadata
total changes: 7213
Successfully write to: output/result50.pickle
Finished Extracting Tracks Data
Finished Extracting Tracks_Metadata
Finished Extracting Recording_Metadata
total changes: 7388
Successfully write to: output/result51.pickle
Finished Extracting Tracks Data
Finished Extracting Tracks_Metadata
Finished Extracting Recording_Metadata
total changes: 7540
Successfully write to: output/result52.pickle
Finished Extracting Tracks Data
Finished Extracting Tracks_Metadata
Finished Extracting Recording_Metadata
total changes: 7748
Successfully write to: output/result53.pickle
Finished Extracting Tracks Data
Finished Extracting Tracks_Metadata
F

#### The stored data takes the folowing format. 
    ([(x,y,x,.,.,.),(.,.,.,.,)],2).
    A tuple containing a list of values and the lane change direction.
    1 for left lane change
    2 for right change and 0 for cars that retain their individual lane
    
